In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# https://www.kaggle.com/eshaandeshpande/cots-detection-with-yolov4-darknet

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import greatbarrierreef
import sys
import cv2 as cv
import os
from tqdm.notebook import tqdm
tqdm.pandas()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cp -R /kaggle/input/darknet-run-environment/darknet /kaggle/working

In [ ]:
%cp '/kaggle/input/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom_final.weights' '/kaggle/working/darknet'
#%cp '/kaggle/input/predconf/run.py' '/kaggle/working/darknet'
%cp '/kaggle/input/cots-yolov4-obj/obj.data' '/kaggle/working/darknet/data'
%cp '/kaggle/input/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom.cfg' '/kaggle/working/darknet/cfg'
%cp '/kaggle/input/cots-yolov4-obj/obj.names' '/kaggle/working/darknet/data'

In [ ]:
%cd /kaggle/working/darknet

!cp '../../input/libcuda/libcuda.so' .

!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile

!sed -i 's/LDFLAGS+= -L\/usr\/local\/cuda\/lib64 -lcuda -lcudart -lcublas -lcurand/LDFLAGS+= -L\/usr\/local\/cuda\/lib64 -lcudart -lcublas -lcurand -L\/kaggle\/working\/darknet -lcuda/' Makefile
!make &> compile.log

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import darknet
import cv2

network, class_names, class_colors = darknet.load_network(
        '/kaggle/working/darknet/cfg/yolov4-custom.cfg',
        '/kaggle/working/darknet/data/obj.data',
        '/kaggle/working/darknet/yolov4-custom_final.weights',
        1
    )


for (pixel_array, sample_prediction_df) in iter_test:
    ip_image = pixel_array[:,:,::-1]   #cv2.imread('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg')
    darknet_image = darknet.make_image(416, 416, 3)
    ip_rgb = cv2.cvtColor(ip_image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(ip_rgb,(416,416) ,cv2.INTER_AREA)
    img_resized[416:416, 0:416] = (0,0,0)
    darknet.copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = darknet.detect_image(network, class_names, darknet_image, 0.35)
    darknet.free_image(darknet_image)
    preds = []
    for label, confidence, bbox in detections:
        x1, y1, w1, h1 = bbox
        if y1 < 416:    
            xmin, ymin, xmax, ymax = darknet.bbox2points(bbox)
            preds.append(f'0.{confidence[0]} {int(xmin*2)} {int(ymin*2)} {int(w1*2)} {int(h1*2)}')
    prediction_str = ' '.join(preds)
    print(prediction_str)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)   # register your predictions
#     ifinal = darknet.draw_boxes(detections, img_resized, class_colors)
#     figure(figsize=(12.8, 12.8))
#     plt.imshow(ifinal)
#     plt.show()
#     cv2.imwrite('/kaggle/working/a.jpg', ifinal)

In [ ]:
import pandas as pd
sub_df = pd.read_csv('/kaggle/working/darknet/submission.csv')
sub_df.head()

In [ ]:
!cp /kaggle/working/darknet/submission.csv /kaggle/working
!rm -r /kaggle/working/darknet

In [ ]:
ROOT_DIR  = '/kaggle/input'
sys.path.append(f'{ROOT_DIR}/tensorflow-great-barrier-reef')
confThreshold = 0.015
confthre = 0.015

# try:
#     net = cv.dnn_DetectionModel(f'{ROOT_DIR}/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom.cfg',
#                                 f'{ROOT_DIR}/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom_final.weights')
#     net.setInputSize(416, 416)
#     net.setInputScale(1.0 / 255)
#     net.setInputSwapRB(True)


#     env = greatbarrierreef.make_env()   # initialize the environment
#     iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
#     #print("iter_test:", iter_test)
    
#     submission_dict = {
#         'id': [],
#         'prediction_string': [],
#     }

#     with open(f'{ROOT_DIR}/cots-yolov4-names/coco.names', 'rt') as f:
#         names = f.read().rstrip('\n').split('\n')


#     for (image_np, sample_prediction_df) in iter_test:
#         bbclasses, scores, bboxes = net.detect(image_np[:,:,::-1], confThreshold=confThreshold, nmsThreshold=0.4)
# #         print('bbclasses:', bbclasses)
# #         print('scores:', scores)
# #         print('bboxes:', bboxes)
#         predictions = []
#         for i in range(len(bboxes)):
#             box = bboxes[i]
#             cls_id = int(bbclasses[i])
#             score = scores[i]
#             if score < confThreshold:
#                 continue
#             x_min = int(box[0])
#             y_min = int(box[1])
#             x_max = int(box[2])
#             y_max = int(box[3])

#             bbox_width = x_max - x_min
#             bbox_height = y_max - y_min

#             predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))

#         prediction_str = ' '.join(predictions)
#         print('Prediction:', prediction_str)
#         sample_prediction_df['annotations'] = prediction_str
#         env.predict(sample_prediction_df)
        
# except Exception as exc:
#     print("Following error:: ", exc)

In [ ]:
def load_model(conf=0.015, iou=0.40):
    net = cv.dnn.readNet(f'{ROOT_DIR}/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom.cfg',
                            f'{ROOT_DIR}/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom_final.weights')
    net = cv.dnn_DetectionModel(net)
    net.setInputParams(size=(416, 416), scale=1/255, swapRB=True)
    return net

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes
def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes
def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes
def yolo2coco(bboxes, image_height=416, image_width=416):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

In [ ]:
IMG_SIZE=416

with open('/kaggle/input/cots-yolov4-names/coco.names', 'rt') as f:
    names = f.read().rstrip('\n').split('\n')
    
def predict(net, img, size=IMG_SIZE):
    confs = []
    bboxes = []
    height, width = img.shape[:2]
    bbclasses, scores, bboxes = net.detect(img, confThreshold=confThreshold, nmsThreshold=0.4)
    if len(bboxes):
        confs=[]
        for i in scores:
            confs.append('{:.2f}'.format(i))
        score=np.array(confs,dtype=float)  
        return bboxes, score
    else:
        return [],[]

In [ ]:
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

In [ ]:
from PIL import Image
from numpy import asarray

img = Image.open('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/1875.jpg')
  
numpydata = asarray(img)

net = load_model(conf=0.015, iou=0.40)

bboxes, confs  = predict(net, numpydata, size=IMG_SIZE)

print(bboxes, confs)

In [ ]:
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()
net = load_model(conf=0.015, iou=0.40)
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    bboxes, confs  = predict(net, img, size=IMG_SIZE)
    annot          = format_prediction(bboxes, confs)
    pred_df['annotations'] = annot
    env.predict(pred_df)

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()